In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

In [2]:
trainData = pd.read_csv('Data/train.csv')
trainLabels = pd.read_csv('Data/train_result.csv')

In [3]:
trainData[trainData < 0] = 0
#Réduit de 73 features
trainData = trainData.loc[:, (trainData != 0).any(axis=0)]
#Réduit de 237 features
trainData = trainData.loc[:, (trainData > 0.95).any(axis=0)]

In [117]:
from codecs import ignore_errors


class LogisiticRegression:
    def __init__(self, trainData, trainLabels):
        self.trainData = trainData
        self.trainLabels = trainLabels
        self.featureSize = len(trainData.columns)
        self.trainSize = len(self.trainData.index)
        self.labels = np.sort(trainLabels['Class'].unique())
        self.labelSize = self.labels.size

        self.weights = pd.DataFrame(np.zeros((self.featureSize,self.labelSize)))
        self.bias = pd.Series(np.zeros(self.labelSize))

    def sigmoid(self,df):
        result = np.zeros(len(df))

        for i,val in enumerate(df):
            result[i] = np.exp(val)/np.sum(np.exp(df))

        return result

    def oneHotEncoder(self,Y):
        #Transformer les labels en one hot pour calculer le loss
        df = pd.DataFrame(np.zeros((self.labelSize,Y['Class'].size)))
        for i,val in enumerate(Y['Class']):
            ind = self.labels[self.labels==val]
            df.loc[val,i] = 1
        return df
    
    def predictSoftmax(self, df):
        #Calculer le le produit de W^T*X^T+b pour tous les echantillons
        newW = self.weights.transpose()
        newX = df.transpose()
        wx = newW.dot(newX.to_numpy())
        gFunction = wx.transpose()+self.bias

        #Faire le argmax(softmax(g(x))) pour tous les echantillons
        softMaxResult = gFunction.apply(self.sigmoid,axis=0)

        return softMaxResult

    def predict(self, df):
        #Calculer le le produit de W^T*X^T+b pour tous les echantillons
        newW = self.weights.transpose()
        newX = df.transpose()
        wx = newW.dot(newX.to_numpy())
        gFunction = wx.transpose()+self.bias

        #Faire le argmax(softmax(g(x))) pour tous les echantillons
        softMaxResult = gFunction.apply(self.sigmoid,axis=0)
        predictions = softMaxResult.idxmax(axis=1)
        predictionLabels = [self.labels[i] for i in predictions]

        return predictionLabels
    
    def loss(self,Y,Yhat,batchSize):
        #retourner la cout (entropie croise) 
        return (-np.sum(np.log(np.sum(Y.mul(Yhat.transpose()),axis=0))))/batchSize

    def gradient(self,X,Y,Yhat,lam,bathSize):
        #retourner le gradient avec une regularisation L2 
        #Ajouter le biais au X et W pour avoir son gradient
        X = pd.concat(X.transpose(),pd.Series(1,index=X.index),ignore_index=True)
        W = pd.concat(self.weights,pd.Series(1,index=range(self.labelSize)),ignore_index=True)
        return (X.dot((Y-Yhat.transpose()).transpose().to_numpy()))/bathSize + 2*lam*W.to_numpy()

    def train(self, iter, batchSize, lam, alpha):
        loss = pd.Series(np.zeros(iter))

        for i in range(iter):
            #Creer aleatoire la batch 
            sampleTrain = trainData.sample(batchSize,replace=False,axis=0)
            sampleLabels = trainLabels.iloc[sampleTrain.index,:]
            yOneHot = self.oneHotEncoder(sampleLabels)
            
            #Predire les probabilite des labels et calculer le cout
            predictionProb = self.predictSoftmax(sampleTrain)
            loss[i] = self.loss(yOneHot,predictionProb,batchSize)

            #Calculer le gradient
            grad = self.gradient(sampleTrain,yOneHot,predictionProb,lam,batchSize)

            #Ameliorer W et B
            self.weights = self.weights + alpha*grad[:-1,:]
            self.bias = self.bias + alpha*grad[-1,:]
            
        return grad
        

In [116]:
logReg = LogisiticRegression(trainData,trainLabels)
x = logReg.train(1,12,0.8,0.1)

AttributeError: 'DataFrame' object has no attribute 'concat'